# Demand-Based Pricing Optimization

### Intro.

#### Context
Have you ever wondered when the best time of year to book a hotel room is? Or the optimal length of stay in order to get the best daily rate? What if you wanted to predict whether or not a hotel was likely to receive a disproportionately high number of special requests?

This hotel booking dataset can help you explore those questions!

Content
This data set contains booking information for a city hotel and a resort hotel, and includes information such as when the booking was made, length of stay, the number of adults, children, and/or babies, and the number of available parking spaces, among other things.

All personally identifying information has been removed from the data.

#### What is Revenue Management
There is a key question:

    - What price should I charge for my available supply?
Revenue management is a business approach to balance supply and demand to maximize revenue.

Revenue managers have two strategic levers they can use to optimize revenue:

    - Capacity control: How many rooms to allocate to each segment
    - Pricing: What price to charge for each segment

If your hotel is one of the many that does not have a revenue management system, this workbook will provide you a simple model, that you can use as a baseline to making better pricing decisions.

#### Price Modeling and Optimal pricing



### imports

In [4]:
import kagglehub
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
import datetime

%matplotlib inline


### download dataset

In [6]:
# Download latest version
path = kagglehub.dataset_download("jessemostipak/hotel-booking-demand")
print("Path to dataset files:", path)

def move_dataset_files(path: str):
    source_dir = f"{path}"
    project_root = os.path.split(os.getcwd())[0] # Assumes the script is run from the project root
    destination_dir = os.path.join(project_root, "data")

    try:
        # Create the destination directory if it doesn't exist
        os.makedirs(destination_dir, exist_ok=True)
        print(f"Ensured destination directory exists: {destination_dir}")

        # Check if the source directory exists
        if not os.path.isdir(source_dir):
            print(f"Error: Source directory not found at {source_dir}")
            return

        # List all files in the source directory
        files_to_move = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

        if not files_to_move:
            print(f"No files found in the source directory: {source_dir}")
            return

        print(f"Found {len(files_to_move)} files to move.")

        # Move each file
        for filename in files_to_move:
            source_file_path = os.path.join(source_dir, filename)
            destination_file_path = os.path.join(destination_dir, filename)
            
            # In case a file with the same name already exists in the destination
            if os.path.exists(destination_file_path):
                print(f"File {filename} already exists in {destination_dir}. Skipping.")
                continue
            
            shutil.move(source_file_path, destination_file_path)
            print(f"Moved: {filename} to {destination_dir}")

        print("Dataset moving process completed.")

    except Exception as e:
        print(f"An error occurred: {e}")

move_dataset_files(path)

100%|██████████| 1.25M/1.25M [00:00<00:00, 3.81MB/s]

Extracting files...
Path to dataset files: C:\Users\yezer\.cache\kagglehub\datasets\jessemostipak\hotel-booking-demand\versions\1
Ensured destination directory exists: c:\Users\yezer\Documents\Projects\dynamic-pricing\data
Found 1 files to move.
Moved: hotel_bookings.csv to c:\Users\yezer\Documents\Projects\dynamic-pricing\data
Dataset moving process completed.


### load data


In [2]:
project_root = os.path.split(os.getcwd())[0] # Assumes the script is run from the project root
data_dir = os.path.join(project_root, "data")
df = pd.read_csv(os.path.join(data_dir, "hotel_bookings.csv"))
print(df.shape)
df.head()

(119390, 32)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03
